# Spring-Mass Systems

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from numba import jit as execute_quickly
%matplotlib inline

/home/andreas/src/env-3.6/lib/python3.6/site-packages/numba/errors.py:102: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [15]:
system = {}
nmasses = 30
for i in range(1, nmasses-1):
    system[i] = [
        (i-1, 3e7, 1/nmasses),
        (i+1, 3e7, 1/nmasses),
    ]

masses = np.ones(nmasses)

zero_if_fixed = np.ones(nmasses)
zero_if_fixed[0] = 0
zero_if_fixed[-1] = 0

positions = np.array([
    np.linspace(0, 1, nmasses),
    np.zeros(nmasses)
])
velocities = np.array([
    np.zeros(nmasses),
    np.zeros(nmasses),
])

dt = 2e-5
gravity = 9.81

In [13]:
ntimes = 10000

pos_in_time = np.zeros((ntimes, 2, nmasses))
vel_in_time = np.zeros((ntimes, 2, nmasses))

pos_in_time[0] = positions
vel_in_time[0] = velocities

for i in range(1, ntimes):
    if i % 1000 == 0:
        print(i)
    force = np.zeros((2, nmasses))
    force[1] = -gravity*masses

    for imass in range(nmasses):
        for jmass, spring_const, rest_length in system.get(imass, []):
            distvec = pos_in_time[i-1, :, jmass] - pos_in_time[i-1, :, imass]
            dist = np.sqrt(np.sum(distvec**2))

            force[:, imass] += distvec * spring_const * (dist-rest_length)

    pos_in_time[i] = pos_in_time[i-1] + dt*vel_in_time[i-1]*zero_if_fixed
    vel_in_time[i] = vel_in_time[i-1] + dt*force/masses*zero_if_fixed

1000
2000
3000
4000
5000
6000
7000
8000
9000


In [14]:
# inscrutable video rendering mumbo-jumbo - execute to see video
# This is yucky and not intended for the students to mess with/understand.

nvideo_frames = 100
frame_step = ntimes // nvideo_frames

fig = plt.figure(figsize=(10, 7))

points, = plt.plot(pos_in_time[0, 0], pos_in_time[0, 1], "o")
    
def animate(i):
    points.set_data((pos_in_time[i*frame_step, 0], pos_in_time[i*frame_step, 1]))
    return (points,)

anim = animation.FuncAnimation(
    fig, animate, frames=nvideo_frames, interval=50, blit=True)

from IPython.display import HTML
html = HTML(anim.to_html5_video())
plt.clf()

html

<Figure size 720x504 with 0 Axes>